In [1]:
import numpy as np
import random
import csv, sqlite3
import logging
import cProfile
import lazypandas
import matplotlib.pyplot as plt
import datetime, scipy 
import datetime, warnings, scipy 
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
lz = lazypandas.lazy_pandas()

LZ Creating engine


In [3]:
airlines = lz.read_csv('../dot/airlines.csv')
airport = lz.read_csv('../dot/airports.csv')
data  = lz.read_csv('../dot/red_flights.csv')

table name airlines
new_stack
139628822623456
new lzpd stack {'airlines': ['IATA_CODE', 'AIRLINE', 'auto_index']} select airlines.IATA_CODE, airlines.AIRLINE, airlines.auto_index
from airlines

table name airports
new_stack
139628776829112
new lzpd stack {'airports': ['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE', 'auto_index']} select airports.IATA_CODE, airports.AIRPORT, airports.CITY, airports.STATE, airports.COUNTRY, airports.LATITUDE, airports.LONGITUDE, airports.auto_index
from airports

table name red_flights
new_stack
139628776917984
new lzpd stack {'red_flights': ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON

In [4]:
data.info()

lazyDataFrame
tables
  red_flights :
    YEAR : REAL
    MONTH : REAL
    DAY : REAL
    DAY_OF_WEEK : REAL
    AIRLINE : TEXT
    FLIGHT_NUMBER : REAL
    TAIL_NUMBER : TEXT
    ORIGIN_AIRPORT : TEXT
    DESTINATION_AIRPORT : TEXT
    SCHEDULED_DEPARTURE : REAL
    DEPARTURE_TIME : REAL
    DEPARTURE_DELAY : REAL
    TAXI_OUT : REAL
    WHEELS_OFF : REAL
    SCHEDULED_TIME : REAL
    ELAPSED_TIME : REAL
    AIR_TIME : REAL
    DISTANCE : REAL
    WHEELS_ON : REAL
    TAXI_IN : REAL
    SCHEDULED_ARRIVAL : REAL
    ARRIVAL_TIME : REAL
    ARRIVAL_DELAY : REAL
    DIVERTED : REAL
    CANCELLED : REAL
    CANCELLATION_REASON : TEXT
    AIR_SYSTEM_DELAY : REAL
    SECURITY_DELAY : REAL
    AIRLINE_DELAY : REAL
    LATE_AIRCRAFT_DELAY : REAL
    WEATHER_DELAY : REAL
    auto_index : INTEGER



In [5]:
data.shape

(99999, 32)

In [6]:
data.describe()

,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,auto_index
count,99999.0,99999.0,99999.000000,99999.000000,99999.000000,99999.000000,97701.000000,97701.000000,97628.000000,97628.000000,...,97559.000000,97386.000000,99999.000000,99999.000000,34625.000000,34625.000000,34625.000000,34625.000000,34625.000000,99999.000000
mean,2015.0,1.0,3.733887,4.107321,2240.849648,1306.350254,1326.267930,19.938046,17.155908,1348.428514,...,1458.032442,18.342575,0.002240,0.023890,14.026801,0.070787,17.853921,26.014556,2.751971,49999.000000
std,0.0,0.0,1.815522,2.098459,1818.708843,482.327514,503.511173,45.843719,10.738428,504.144849,...,538.213952,48.878412,0.047276,0.152708,25.965405,1.692526,43.527539,42.593343,17.265760,28867.369122
min,2015.0,1.0,1.000000,1.000000,1.000000,5.000000,1.000000,-39.000000,1.000000,1.000000,...,1.000000,-65.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.0,1.0,2.000000,2.000000,759.000000,904.000000,911.000000,-3.000000,11.000000,928.000000,...,1050.000000,-8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24999.500000
50%,2015.0,1.0,4.000000,4.000000,1693.000000,1300.000000,1317.000000,3.000000,14.000000,1330.000000,...,1455.000000,4.000000,0.000000,0.000000,4.000000,0.000000,4.000000,10.000000,0.000000,49999.000000
75%,2015.0,1.0,5.000000,6.000000,3394.000000,1720.000000,1739.000000,25.000000,20.000000,1753.000000,...,1911.000000,27.000000,0.000000,0.000000,19.000000,0.000000,19.000000,35.000000,0.000000,74998.500000
max,2015.0,1.0,7.000000,7.000000,7438.000000,2359.000000,2400.000000,1380.000000,176.000000,2400.000000,...,2400.000000,1384.000000,1.000000,1.000000,590.000000,107.000000,1380.000000,891.000000,562.000000,99998.000000


In [7]:
data.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,auto_index
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,-22.0,0,0,None,None,None,None,None,None,0
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,-9.0,0,0,None,None,None,None,None,None,1
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,5.0,0,0,None,None,None,None,None,None,2
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,-9.0,0,0,None,None,None,None,None,None,3
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,-21.0,0,0,None,None,None,None,None,None,4


In [8]:
data.tail() ## wrong order and index number

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,auto_index
0,2015,1,7,3,UA,1135,N38473,ORD,MCO,1109,...,-1.0,0,0,None,None,None,None,None,None,99998
1,2015,1,7,3,OO,5606,N582SW,SFO,BFL,1108,...,7.0,0,0,None,None,None,None,None,None,99997
2,2015,1,7,3,NK,394,N522NK,LAS,PHL,1108,...,2.0,0,0,None,None,None,None,None,None,99996
3,2015,1,7,3,EV,4992,N837AS,ATL,BQK,1108,...,-11.0,0,0,None,None,None,None,None,None,99995
4,2015,1,7,3,OO,4774,N463SW,SLC,PSC,1107,...,-4.0,0,0,None,None,None,None,None,None,99994


In [9]:
data

[[2015 1 1 ... None None 0]
 [2015 1 1 ... None None 1]
 [2015 1 1 ... None None 2]
 ...
 [2015 1 7 ... None None 99996]
 [2015 1 7 ... None None 99997]
 [2015 1 7 ... None None 99998]]

In [10]:
airport = airport.dropna(subset = ['LATITUDE','LONGITUDE'])

caught drop na subset () {'subset': ['LATITUDE', 'LONGITUDE']}
new lzpd stack {'airports': ['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE', 'auto_index']} select airports.IATA_CODE, airports.AIRPORT, airports.CITY, airports.STATE, airports.COUNTRY, airports.LATITUDE, airports.LONGITUDE, airports.auto_index
from airports
where (LATITUDE is not null) and (LONGITUDE is not null)



In [11]:
airport.head(10)

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,auto_index
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040,0
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190,1
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919,2
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183,3
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447,4
5,ACK,Nantucket Memorial Airport,Nantucket,MA,USA,41.25305,-70.06018,5
6,ACT,Waco Regional Airport,Waco,TX,USA,31.61129,-97.23052,6
7,ACV,Arcata Airport,Arcata/Eureka,CA,USA,40.97812,-124.10862,7
8,ACY,Atlantic City International Airport,Atlantic City,NJ,USA,39.45758,-74.57717,8
9,ADK,Adak Airport,Adak,AK,USA,51.87796,-176.64603,9


In [12]:
airlines

[['UA' 'United Air Lines Inc.' '0']
 ['AA' 'American Airlines Inc.' '1']
 ['US' 'US Airways Inc.' '2']
 ['F9' 'Frontier Airlines Inc.' '3']
 ['B6' 'JetBlue Airways' '4']
 ['OO' 'Skywest Airlines Inc.' '5']
 ['AS' 'Alaska Airlines Inc.' '6']
 ['NK' 'Spirit Air Lines' '7']
 ['WN' 'Southwest Airlines Co.' '8']
 ['DL' 'Delta Air Lines Inc.' '9']
 ['EV' 'Atlantic Southeast Airlines' '10']
 ['HA' 'Hawaiian Airlines Inc.' '11']
 ['MQ' 'American Eagle Airlines Inc.' '12']
 ['VX' 'Virgin America' '13']]

In [13]:
# Dropping of subset of null values
data1 = data.dropna(subset = ["TAIL_NUMBER",'DEPARTURE_TIME','DEPARTURE_DELAY','TAXI_OUT','WHEELS_OFF','SCHEDULED_TIME',
             'ELAPSED_TIME','AIR_TIME','WHEELS_ON','TAXI_IN','ARRIVAL_TIME','ARRIVAL_DELAY'])

caught drop na subset () {'subset': ['TAIL_NUMBER', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'WHEELS_ON', 'TAXI_IN', 'ARRIVAL_TIME', 'ARRIVAL_DELAY']}
new lzpd stack {'red_flights': ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'auto_index']} select red_flights.YEAR, red_flights.MONTH, red_flights.DAY, red_flights.DAY_OF_WEEK, red_flights.AIRLINE, red_flights.FLIGHT_NUMBER, red_flights.TAIL_NUMBER, red_flights.ORIGIN_AIRPORT, red_flights.DESTINATION_AIRPORT, red_flights.SCHEDULED_DEPA

In [14]:
data1.shape

(97386, 32)

In [15]:
# Creting Dataset w.r.t different Types of Delays
data11 = data1.dropna(subset = ['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'])
data11 = data11.drop(['YEAR','MONTH','DAY','DAY_OF_WEEK','TAIL_NUMBER','SCHEDULED_DEPARTURE','DEPARTURE_TIME','SCHEDULED_TIME',
                     'SCHEDULED_ARRIVAL','ARRIVAL_TIME','DIVERTED','CANCELLED','CANCELLATION_REASON','FLIGHT_NUMBER','WHEELS_OFF',
                     'WHEELS_ON','AIR_TIME'],axis = 1)

caught drop na subset () {'subset': ['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']}
new lzpd stack {'red_flights': ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'auto_index']} select red_flights.YEAR, red_flights.MONTH, red_flights.DAY, red_flights.DAY_OF_WEEK, red_flights.AIRLINE, red_flights.FLIGHT_NUMBER, red_flights.TAIL_NUMBER, red_flights.ORIGIN_AIRPORT, red_flights.DESTINATION_AIRPORT, red_flights.SCHEDULED_DEPARTURE, red_flights.DEPARTURE_TIME, red_flights.DEPARTURE_DELAY, red_flights.TAXI_OUT, 

In [16]:
data11.info()

lazyDataFrame
tables
  red_flights :
    YEAR : REAL
    MONTH : REAL
    DAY : REAL
    DAY_OF_WEEK : REAL
    AIRLINE : TEXT
    FLIGHT_NUMBER : REAL
    TAIL_NUMBER : TEXT
    ORIGIN_AIRPORT : TEXT
    DESTINATION_AIRPORT : TEXT
    SCHEDULED_DEPARTURE : REAL
    DEPARTURE_TIME : REAL
    DEPARTURE_DELAY : REAL
    TAXI_OUT : REAL
    WHEELS_OFF : REAL
    SCHEDULED_TIME : REAL
    ELAPSED_TIME : REAL
    AIR_TIME : REAL
    DISTANCE : REAL
    WHEELS_ON : REAL
    TAXI_IN : REAL
    SCHEDULED_ARRIVAL : REAL
    ARRIVAL_TIME : REAL
    ARRIVAL_DELAY : REAL
    DIVERTED : REAL
    CANCELLED : REAL
    CANCELLATION_REASON : TEXT
    AIR_SYSTEM_DELAY : REAL
    SECURITY_DELAY : REAL
    AIRLINE_DELAY : REAL
    LATE_AIRCRAFT_DELAY : REAL
    WEATHER_DELAY : REAL
    auto_index : INTEGER



In [17]:
# The other Dataset
Flight_Delays = data11

In [18]:
# Creating Dataset by removing null values by not focussing fully on different types of Delays
data2 = data1.drop(['CANCELLATION_REASON','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY',
                    'LATE_AIRCRAFT_DELAY','WEATHER_DELAY'],axis = 1)

caught drop  labels (['CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],) {'axis': 1}
new lzpd stack {'red_flights': ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'auto_index']} select red_flights.YEAR, red_flights.MONTH, red_flights.DAY, red_flights.DAY_OF_WEEK, red_flights.AIRLINE, red_flights.FLIGHT_NUMBER, red_flights.TAIL_NUMBER, red_flights.ORIGIN_AIRPORT, red_flights.DESTINATION_AIRPORT, red_flights.SCHEDULED_DEPARTURE, red_flights.DEPARTURE_TIME, red_flights.DEPARTURE_DELAY, r

In [19]:
data2.shape

(97386, 26)

In [20]:
data2.info()

lazyDataFrame
tables
  red_flights :
    YEAR : REAL
    MONTH : REAL
    DAY : REAL
    DAY_OF_WEEK : REAL
    AIRLINE : TEXT
    FLIGHT_NUMBER : REAL
    TAIL_NUMBER : TEXT
    ORIGIN_AIRPORT : TEXT
    DESTINATION_AIRPORT : TEXT
    SCHEDULED_DEPARTURE : REAL
    DEPARTURE_TIME : REAL
    DEPARTURE_DELAY : REAL
    TAXI_OUT : REAL
    WHEELS_OFF : REAL
    SCHEDULED_TIME : REAL
    ELAPSED_TIME : REAL
    AIR_TIME : REAL
    DISTANCE : REAL
    WHEELS_ON : REAL
    TAXI_IN : REAL
    SCHEDULED_ARRIVAL : REAL
    ARRIVAL_TIME : REAL
    ARRIVAL_DELAY : REAL
    DIVERTED : REAL
    CANCELLED : REAL
    CANCELLATION_REASON : TEXT
    AIR_SYSTEM_DELAY : REAL
    SECURITY_DELAY : REAL
    AIRLINE_DELAY : REAL
    LATE_AIRCRAFT_DELAY : REAL
    WEATHER_DELAY : REAL
    auto_index : INTEGER



In [21]:
data2['DEPARTURE_TIME']

caught get [ DEPARTURE_TIME ]
col_name DEPARTURE_TIME
simple project DEPARTURE_TIME
new lzpd stack {'red_flights': ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'auto_index']} select DEPARTURE_TIME
from red_flights
where (TAIL_NUMBER is not null) and (DEPARTURE_TIME is not null) and (DEPARTURE_DELAY is not null) and (TAXI_OUT is not null) and (WHEELS_OFF is not null) and (SCHEDULED_TIME is not null) and (ELAPSED_TIME is not null) and (AIR_TIME is not null) and (WHEELS_ON is not null) and (TAXI_IN is not null) and (ARRIVAL_TIME is not null) and (ARRIVA

[2.354e+03 2.000e+00 1.800e+01 ... 1.117e+03 1.101e+03 1.116e+03]

In [22]:
# Creating a function to change the way of representation of time in the column
def Format_Hourmin(hours):
        if hours == 2400:
            hours = 0
        else:
            hours = "{0:04d}".format(int(hours))
            Hourmin = datetime.time(int(hours[0:2]), int(hours[2:4]))
            return Hourmin

In [23]:
data1['DEPARTURE_TIME']

caught get [ DEPARTURE_TIME ]
col_name DEPARTURE_TIME
simple project DEPARTURE_TIME
new lzpd stack {'red_flights': ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'auto_index']} select DEPARTURE_TIME
from red_flights
where (TAIL_NUMBER is not null) and (DEPARTURE_TIME is not null) and (DEPARTURE_DELAY is not null) and (TAXI_OUT is not null) and (WHEELS_OFF is not null) and (SCHEDULED_TIME is not null) and (ELAPSED_TIME is not null) and (AIR_TIME is not null) and (WHEELS_ON is not null) and (TAXI_IN is not null) and (ARRIVAL_TIME is not null) and (ARRIVA

[2.354e+03 2.000e+00 1.800e+01 ... 1.117e+03 1.101e+03 1.116e+03]

In [34]:
data2['Actual_Departure'] = data1['DEPARTURE_TIME'].apply(Format_Hourmin)
data2['Actual_Departure']

caught get [ DEPARTURE_TIME ]
col_name DEPARTURE_TIME
simple project DEPARTURE_TIME
new lzpd stack {'red_flights': ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'auto_index']} select DEPARTURE_TIME
from red_flights
where (TAIL_NUMBER is not null) and (DEPARTURE_TIME is not null) and (DEPARTURE_DELAY is not null) and (TAXI_OUT is not null) and (WHEELS_OFF is not null) and (SCHEDULED_TIME is not null) and (ELAPSED_TIME is not null) and (AIR_TIME is not null) and (WHEELS_ON is not null) and (TAXI_IN is not null) and (ARRIVAL_TIME is not null) and (ARRIVA

OperationalError: near ":09": syntax error

In [25]:
data2.info()

lazyDataFrame
tables
  red_flights :
    YEAR : REAL
    MONTH : REAL
    DAY : REAL
    DAY_OF_WEEK : REAL
    AIRLINE : TEXT
    FLIGHT_NUMBER : REAL
    TAIL_NUMBER : TEXT
    ORIGIN_AIRPORT : TEXT
    DESTINATION_AIRPORT : TEXT
    SCHEDULED_DEPARTURE : REAL
    DEPARTURE_TIME : REAL
    DEPARTURE_DELAY : REAL
    TAXI_OUT : REAL
    WHEELS_OFF : REAL
    SCHEDULED_TIME : REAL
    ELAPSED_TIME : REAL
    AIR_TIME : REAL
    DISTANCE : REAL
    WHEELS_ON : REAL
    TAXI_IN : REAL
    SCHEDULED_ARRIVAL : REAL
    ARRIVAL_TIME : REAL
    ARRIVAL_DELAY : REAL
    DIVERTED : REAL
    CANCELLED : REAL
    CANCELLATION_REASON : TEXT
    AIR_SYSTEM_DELAY : REAL
    SECURITY_DELAY : REAL
    AIRLINE_DELAY : REAL
    LATE_AIRCRAFT_DELAY : REAL
    WEATHER_DELAY : REAL
    auto_index : INTEGER

  auto_generated_table_Actual_Departure :
    Actual_Departure : TEXT
    auto_index : INTEGER



In [26]:
# Applying the function to required variables in the dataset
data2['Actual_Departure'] =data1['DEPARTURE_TIME'].apply(Format_Hourmin)
data2['Scheduled_Arrival'] =data1['SCHEDULED_ARRIVAL'].apply(Format_Hourmin)
data2['Scheduled_Departure'] =data1['SCHEDULED_DEPARTURE'].apply(Format_Hourmin)
data2['Actual_Arrival'] =data1['ARRIVAL_TIME'].apply(Format_Hourmin)


caught get [ DEPARTURE_TIME ]
col_name DEPARTURE_TIME
simple project DEPARTURE_TIME
new lzpd stack {'red_flights': ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'auto_index']} select DEPARTURE_TIME
from red_flights
where (TAIL_NUMBER is not null) and (DEPARTURE_TIME is not null) and (DEPARTURE_DELAY is not null) and (TAXI_OUT is not null) and (WHEELS_OFF is not null) and (SCHEDULED_TIME is not null) and (ELAPSED_TIME is not null) and (AIR_TIME is not null) and (WHEELS_ON is not null) and (TAXI_IN is not null) and (ARRIVAL_TIME is not null) and (ARRIVA

OperationalError: near ":09": syntax error

In [ ]:
airports = airports.dropna(subset = ['LATITUDE','LONGITUDE'])

In [ ]:
data1 = data.dropna(subset = ["TAIL_NUMBER",'DEPARTURE_TIME','DEPARTURE_DELAY','TAXI_OUT','WHEELS_OFF','SCHEDULED_TIME',
             'ELAPSED_TIME','AIR_TIME','WHEELS_ON','TAXI_IN','ARRIVAL_TIME','ARRIVAL_DELAY'])

In [ ]:
data11 = data1.dropna(subset = ['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'])
data11 = data11.drop(['YEAR','MONTH','DAY','DAY_OF_WEEK','TAIL_NUMBER','SCHEDULED_DEPARTURE','DEPARTURE_TIME','SCHEDULED_TIME',
                     'SCHEDULED_ARRIVAL','ARRIVAL_TIME','DIVERTED','CANCELLED','CANCELLATION_REASON','FLIGHT_NUMBER','WHEELS_OFF',
                     'WHEELS_ON','AIR_TIME'],axis = 1)

In [ ]:
# The other Dataset
Flight_Delays = data11

In [ ]:
# Creating Dataset by removing null values by not focussing fully on different types of Delays
data2 = data1.drop(['CANCELLATION_REASON','AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY',
                    'LATE_AIRCRAFT_DELAY','WEATHER_DELAY'],axis = 1)

In [ ]:
data2.shape

In [ ]:
data2.info()

In [ ]:
# Creating a function to change the way of representation of time in the column
@np.vectorize
def Format_Hourmin(hours):
        if hours == 2400:
            hours = 0
        else:
            hours = "{0:04d}".format(int(hours))
            Hourmin = datetime.time(int(hours[0:2]), int(hours[2:4]))
            return Hourmin

In [ ]:
data1['CANCELLED']

In [ ]:
out = data1.execute('select month, airline, AVG(CANCELLED) from red_flights group by airline, month ').fetchall()
m, a, rel = list(map(list, zip(*out)))


In [ ]:
print(a)
plt.plot(a, rel)
plt.show()

In [ ]:
x = data1['DEPARTURE_TIME'].apply(Format_Hourmin)
#data2['Actual_Departure'] = x
x

In [ ]:
plt.plot(data['ELAPSED_TIME'].array, data['DISTANCE'].array, '.')
plt.show()

In [ ]:
data = data.dropna(['DISTANCE', 'ARRIVAL_DELAY'])
type(a)

In [ ]:
x = data['DISTANCE'].to_pd().values.flatten()
y = data['ARRIVAL_DELAY'].to_pd().values.flatten()

heatmap, xedges, yedges = np.histogram2d(x, y, bins=16)
plt.imshow(heatmap.T, origin='lower')
plt.show()

In [ ]:
a = data['DISTANCE']
a.generate_stack()

In [ ]:
from util import csvToDb, out_to_pd
out = data.execute('select min(AIRLINE_DELAY) from red_flights')
print(out.description[0])
out.fetchone()
#data.dotted_datatype

In [ ]:
a = data.to_pd()
a.describe()

In [ ]:
data.info()

In [ ]:
# Merging on AIRLINE and IATA_CODE
data2 = data2.merge(airlines, left_on='AIRLINE', right_on='IATA_CODE', how='inner')

In [ ]:
data2['airlines.AIRLINE']